# Genetic Algorithms

### Idea of the algorithm:
1. Using a genetic algorithm, build the best schedule based on the demand for each individual day;
2. If there is still enough time to produce new mixes on the current day, iteratively as long as there is such a possibility, at each step the sequence of mixes of the next day is chosen, which transfering will give the biggest improvement, which is calculated using the best fitness score for the new GA run. Since transferring the mixes that were prepared on both days will cause the least waste of time on the previous day, first one will determine the overlaps in the preparation of mixes for both days;
3. If at any day the time spent exceeds the duration of the day, the appropriate number of mixes is removed from that day to compensate the time spending.

### Step 1: Data preprocessing

In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
# Exporting the data
df = pd.read_excel('GA matrix.xlsx')
df

,ID,Unnamed: 1,1,2,3,4,9,10,13,14,16,22,26,27
0,NaN,name,SM,ST,DG,HC,PS,ICT,CSP,LGS,VM,CMS,GS,CSM
1,1,SM,0,1,1,3,1,3,3,1,3,3,1,1
2,2,ST,9,0,1,3,9,3,3,1,3,3,1,9
3,3,DG,9,3,0,3,9,3,3,3,3,3,1,9
4,4,HC,9,9,9,0,9,3,3,9,9,3,9,9
5,9,PS,9,9,9,1,0,1,9,9,9,9,9,9
6,10,ICT,9,9,9,1,9,0,9,9,9,9,9,9
7,13,CSP,9,9,9,1,9,1,0,9,9,3,9,9
8,14,LGS,9,9,9,9,9,9,9,0,9,9,9,9
9,16,VM,9,1,1,1,9,3,1,1,0,3,1,9


In [3]:
# Supporting variables for converting indexes from table with data into sequential integers
spices_indexes = [1, 2, 3, 4, 9, 10, 13, 14, 16, 22, 26, 27]
cleaning_class = {0:0, 1:20, 3:60, 9:180}
SPICES_NUM = 12

In [4]:
# Transition matrix from one mix to another with the transition class
transition_matrix = pd.DataFrame(df.shift(-1).iloc[0:SPICES_NUM].loc[:, spices_indexes])
for i, column in enumerate(transition_matrix.columns):
    transition_matrix.rename(columns={column: i}, inplace=True)
transition_matrix

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,1,1,3,1,3,3,1,3,3,1,1
1,9,0,1,3,9,3,3,1,3,3,1,9
2,9,3,0,3,9,3,3,3,3,3,1,9
3,9,9,9,0,9,3,3,9,9,3,9,9
4,9,9,9,1,0,1,9,9,9,9,9,9
5,9,9,9,1,9,0,9,9,9,9,9,9
6,9,9,9,1,9,1,0,9,9,3,9,9
7,9,9,9,9,9,9,9,0,9,9,9,9
8,9,1,1,1,9,3,1,1,0,3,1,9
9,9,9,9,3,3,3,3,9,9,0,9,9


In [5]:
# Transition matrix from one mix to another with the time in minutes
for i in range(SPICES_NUM):
    for j in range(SPICES_NUM):
        transition_matrix.at[i,j] = cleaning_class[transition_matrix.iloc[i][j]]
transition_matrix

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,20,20,60,20,60,60,20,60,60,20,20
1,180,0,20,60,180,60,60,20,60,60,20,180
2,180,60,0,60,180,60,60,60,60,60,20,180
3,180,180,180,0,180,60,60,180,180,60,180,180
4,180,180,180,20,0,20,180,180,180,180,180,180
5,180,180,180,20,180,0,180,180,180,180,180,180
6,180,180,180,20,180,20,0,180,180,60,180,180
7,180,180,180,180,180,180,180,0,180,180,180,180
8,180,20,20,20,180,60,20,20,0,60,20,180
9,180,180,180,60,60,60,60,180,180,0,180,180


In [6]:
# Demand matrix on the whole week
spices_indexes_copy = spices_indexes.copy()[:7]
spices_indexes_copy.insert(0, 'ID')
demand = df.shift(-23).iloc[0:SPICES_NUM].loc[:, spices_indexes_copy]
demand = demand.sort_values(by=['ID']).reset_index(drop = True).drop('ID', axis=1)
for i, column in enumerate(demand.columns):
    demand.rename(columns={column: i}, inplace=True)
demand

,0,1,2,3,4,5,6
0,0,0,0,0,0,7,0
1,7,0,0,0,0,0,0
2,12,12,12,12,12,7,0
3,0,4,0,0,0,0,0
4,0,0,5,5,0,0,0
5,0,6,6,6,6,3,6
6,0,0,0,0,0,6,14
7,5,5,5,5,5,0,5
8,10,10,10,10,10,10,0
9,0,0,0,0,5,0,5


In [7]:
# Demand matrix on the whole week in form of array
days_array = []
for i in demand.columns:
    temp = []
    for j in range(SPICES_NUM):
        temp.append(demand.iloc[j][i])
    days_array.append(temp)
days_array

[[0, 7, 12, 0, 0, 0, 0, 5, 10, 0, 10, 0],
 [0, 0, 12, 4, 0, 6, 0, 5, 10, 0, 10, 10],
 [0, 0, 12, 0, 5, 6, 0, 5, 10, 0, 10, 10],
 [0, 0, 12, 0, 5, 6, 0, 5, 10, 0, 10, 0],
 [0, 0, 12, 0, 0, 6, 0, 5, 10, 5, 10, 0],
 [7, 0, 7, 0, 0, 3, 6, 0, 10, 0, 5, 10],
 [0, 0, 0, 0, 0, 6, 14, 5, 0, 5, 10, 10]]

In [8]:
def intersecting_spices(day1, day2):
    """
    Compute overlaps of mixes for two days

    :param day1: demand for the first day
    :param day2: demand for the second day
    :return: list of mixes overlaps
    """
    intersect = []
    for i in range(len(day1)):
        if day1[i]!=0 and day2[i]!=0:
            intersect.append(day2[i])
        else:
            intersect.append(0)
    return intersect

### Step 2: Developing a genetic algorithm

In [9]:
def crossover(chromosome1, chromosome2, start_position, end_position, demand_mix):
    """
    Executing a crossover operation

    :param chromosome1: list representing the first chromosome
    :param chromosome2: list representing the second chromosome
    :param start_position: index of the start position for crossover
    :param end_position: index of the end position for crossover
    :param demand_mix: demand for the current day
    :return: two lists of the children chromosome 
    """
    child1, child2 = chromosome1.copy(), chromosome2.copy()
    mix1, mix2 = demand_mix.copy(),  demand_mix.copy()
    start_position, end_position = min(start_position,end_position), max(start_position,end_position)

    # Section of the first chromosome block between start and end position
    child1 = child1[start_position:end_position]
    for spice in child1:
         mix1[spice]-=1
    # Appending values from the second chromosome according to the demand
    for spice in chromosome2:
        if mix1[spice]!=0:
            child1.append(spice)
            mix1[spice]-=1
    
    # Section of the second chromosome block between start and end position
    child2 = child2[start_position:end_position]
    for spice in child2:
         mix2[spice]-=1
    # Appending values from the first chromosome according to the demand
    for spice in chromosome1:
        if mix2[spice]!=0:
            child2.append(spice)
            mix2[spice]-=1
    
    return child1, child2

def mutation(chromosome, start_position, end_position):
    """
    Executing a mutation operation

    :param chromosome: list representing the chromosome
    :param start_position: index of the start position for mutation
    :param end_position: index of the end position for mutation
    :return: list of the child chromosome 
    """
    start_position, end_position = min(start_position,end_position), max(start_position,end_position)
    child = chromosome[:start_position] + chromosome[start_position:
                                                     end_position:-1] + chromosome[end_position:]
    return child

def fitness(matrix, chromosome):
    """
    Compute the fitness score for the particular chromosome

    :param matrix: transition matrix for different mixes
    :param chromosome: list representing the chromosome
    :return: fitness score
    """
    # Calculating time on mixing
    mixing_time = len(chromosome)*20
    individual_time = 0
    # Calculating time on cleaning
    for i in range(len(chromosome) - 1):
        individual_time += matrix[chromosome[i + 1]][chromosome[i]]
    # Time remaining after the mixing and cleaning
    remaining_time = 1200 - individual_time - mixing_time
    return remaining_time

def eliterism(matrix, day, population, limitation):
    """
    Computing the best individuals in the population

    :param matrix: transition matrix for two different mixes
    :param day: demand for the current day
    :param population: the population of the chromosomes
    :param limitation: number of individuals in output population
    :return: output population
    """
    fitness_scores = [fitness(matrix, i) for i in population]
    population = list(reversed([x for y, x in sorted(zip(fitness_scores, population))]))
    return population[:limitation]

def generate_population(matrix, number, day):
    """
    Generation of initial populaton

    :param matrix: transition matrix for two different mixes
    :param number: population size
    :param day: demand for the current day
    :return: list of computed quantiles
    """
    # Ideal mix based on the demand
    total_mix = []
    for i in range(len(day)):
        for j in range(int(day[i])):
            total_mix.append(i)
            
    # Generation of the populaton based on ideal mix
    population = []
    while len(population) < number:
        mix = total_mix.copy()
        np.random.shuffle(mix)
        if mix not in population:
            population.append(mix)
    return population

def GA(matrix, day, spices, individuals = 50, generations = 500, keep_parents = False, 
       mutation_chance = 0.1):
    """
    Genetic algorithm

    :param matrix: transition matrix for two different mixes
    :param day: demand for the current day
    :param spices: number of spices
    :param individuals: number of individuals in population
    :param generations: number of generations
    :param keep_parents: keep or remove parents for next generation
    :param mutation_chance: probability of mutatation
    :return: best found schedule and its fitness score
    """
    # Demand for the current day in form of dictionary 
    mix = {}
    for i, spice in enumerate(day):
        mix[i] = spice

    # Generation of initial population and initial fitness
    population = generate_population(matrix, individuals, day)
    first_fitness = fitness(matrix, population[0])
    early_stop = 5
    
    # Repeat for several generations
    for p in range(generations):
        # Fitness of current population
        current_best_fitness = fitness(matrix, population[0])
        # Early stopping createria
        early_stop =  early_stop-1 if first_fitness - current_best_fitness == 0 else 5
        if early_stop == 0:
            break
        first_fitness = current_best_fitness
        
        # Generation of new population
        new_population = []
        if keep_parents:
            new_population.extend(population)

        for i in range(len(population)):
            for j in range(i+1, len(population)):
                # Two random positions for crossover
                start_position = np.random.randint(0, len(population[i]))
                end_position = np.random.randint(start_position, len(population[i]))
                
                # Perform crossover, get children and add them to population
                child1, child2 = crossover(population[i], population[j], 
                                           start_position, end_position, mix)
                new_population.append(child1)
                new_population.append(child2)

            if np.random.rand() < mutation_chance:
                # Two random positions for mutation
                two_positions = set([])
                while len(two_positions) < 2 and len(population[i])!=1:
                    two_positions.add(np.random.randint(0, len(population[i])))
                two_positions = list(two_positions)
                if len(two_positions) == 0:
                    start_position, end_position = two_positions[0], two_positions[1]
                else:
                    start_position, end_position = two_positions[0], two_positions[0]

                # Perform mutation, get child and add it to population
                child = mutation(population[i], start_position, end_position)
                new_population.append(child)
        
        # Get the best individuals from the population of current generation
        population = eliterism(matrix, day, new_population, individuals)
    
    # Get the best individual and its score
    best_mix = eliterism(matrix, day, population, 1)[0]
    offset = fitness(matrix, best_mix)
    return best_mix, offset

### Step 3: Running the genetic algorithm for an individual day

In [32]:
schedule = []
scores = []
# Calculate schedule for each day of the week
for i, day in enumerate(days_array):
    plan, score = GA(transition_matrix, day, SPICES_NUM, individuals = 90, 
                               generations = 150, keep_parents = True, mutation_chance = 0.7)
    schedule.append(plan)
    scores.append(score)

In [33]:
for i, day in enumerate(schedule):
    print("Day", i+1, ":",len(day),"result:",day, "score:",scores[i])

Day 1 : 44 result: [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1, 1, 1, 1, 1, 1, 1, 7, 7, 7, 7, 7] score: 200
Day 2 : 57 result: [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7] score: -260
Day 3 : 58 result: [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5] score: -280
Day 4 : 48 result: [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5] score: -60
Day 5 : 48 result: [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 5, 5, 5, 5, 5, 5] score: -100
Day 6 :

In [34]:
def find_first(array, value):
    """
    Searching for first position of value in the list

    :param array: list of integers
    :param value: element to be found
    :return: position of value in list
    """
    for i in range(len(array)):
        if array[i]==value:
            return i

### Step 4: Transferring mixes from the next day to the previous based on a GA

In [35]:
# Copies of weekly demand, scores and schedule
days_array_copy = copy.deepcopy(days_array)
scores_copy = copy.deepcopy(scores)
schedule_copy = copy.deepcopy(schedule)

# For each pair (current_day, next_day)
for k in range(len(days_array) - 1):
    
    # Calculate overlapping mixes and number of possible transfers
    overlaps = intersecting_spices(days_array_copy[k], days_array_copy[k+1])
    additional_mixes = int(scores_copy[k]//20)
    print("Day:",k + 1)
    demand_of_day = copy.deepcopy(days_array_copy[k+1])
    
    # Until there is enough time in current day
    while additional_mixes > 0:
        best_score = -np.inf
        best_plan = 0
        idx = -1
        transfers_number = 0
        
        # Each non-zero number of overlapping mixes
        for i, inter in enumerate(overlaps):
            if inter != 0:
                if additional_mixes - inter < 0:
                    # If there is not enough space to transfer all mixes
                    demand_copy = copy.deepcopy(demand_of_day)
                    demand_copy[i] = inter - additional_mixes
                    current_transfers = additional_mixes
                else:
                    # If there is enough space to transfer all mixes
                    demand_copy = copy.deepcopy(demand_of_day)
                    demand_copy[i] = 0
                    current_transfers = inter
                
                # Calculate current best plan and score for such a demand
                current_plan, current_score = GA(transition_matrix, demand_copy, SPICES_NUM, individuals = 80, 
                          generations = 150, keep_parents = True, mutation_chance = 0.7)
                
                # If current score is better than the best one
                if current_score > best_score:
                    idx = i
                    best_score = current_score
                    best_plan = current_plan
                    best_demand = demand_copy
                    transfers_number = current_transfers
                    
        # Recalculate demand and score for current and next day
        demand_of_day = best_demand
        days_array_copy[k][idx] += transfers_number
        days_array_copy[k+1][idx] -= transfers_number
        scores_copy[k+1] = best_score
        additional_mixes -= transfers_number
        overlaps[idx] -= transfers_number
        
        # Change plan for current and next day
        first_idx = int(find_first(schedule_copy[k], idx))
        for _ in range(int(transfers_number)):
            schedule_copy[k].insert(first_idx, idx)
        schedule_copy[k+1] = best_plan
        scores_copy[k] -= transfers_number*20

Day: 1
Day: 2
Day: 3
Day: 4
Day: 5
Day: 6


### Step 5: Removing the excessive mixes 

In [36]:
for i in range(len(schedule_copy)):
    # If there is spent to much time at day
    if scores_copy[i] < 0:
        # Number of mixes to be deleted
        offset = (scores_copy[i]//20)
        while offset != 0:
            # Removing extra mixes
            for j in list(set(schedule_copy[i])):
                offset += 1
                scores_copy[i] += 20
                schedule_copy[i].remove(j)
                if offset == 0:
                    break

### Step 6: Output the results 

In [37]:
demand_array = copy.deepcopy(days_array)
# Calculating the number of unproduced mixes
for i in range(len(schedule_copy)):
    uniques = list(set(schedule_copy[i]))
    for unique in uniques:
        demand_array[i][unique] -= schedule_copy[i].count(unique)        
for i in range(len(demand_array)):
    for j in range(len(demand_array[i])):
        if demand_array[i][j] < 0:
            demand_array[i+1][j] += demand_array[i][j]
            demand_array[i][j] -= demand_array[i][j]

# Output the produced number of mixes from the demand and the accuracy of the mixes
total_count = 0
for i in range(len(demand_array)):
    print("Day",i+1,end=": ")
    print()
    counter = 0
    for j in range(len(demand_array[i])):
        if demand_array[i][j]==0:
            if days_array[i][j]!=0:
                print("  Mix",spices_indexes[j],end=": ")
                print(days_array[i][j],end="/")
                print(days_array[i][j])
                counter += days_array[i][j]
            else:
                print("  Mix",spices_indexes[j],end=": ")
                print(days_array[i][j],end="/")
                print(days_array[i][j])
                counter += days_array[i][j]
        else:
            if days_array[i][j]!=0:
                print("  Mix",spices_indexes[j],end=": ")
                print(days_array[i][j]-demand_array[i][j],end="/")
                print(days_array[i][j])
                counter += days_array[i][j]-demand_array[i][j]
            else:
                print("  Mix",spices_indexes[j],end=": ")
                print(days_array[i][j],end="/")
                print(days_array[i][j])
                counter += days_array[i][j]
    print("Accuracy:",100*counter/sum(days_array[i]),end="%")
    print("\n")
    total_count += counter
print("Total accuracy:",100*total_count/sum([sum(i) for i in days_array]),end="%")

Day 1: 
  Mix 1: 0/0
  Mix 2: 7/7
  Mix 3: 12/12
  Mix 4: 0/0
  Mix 9: 0/0
  Mix 10: 0/0
  Mix 13: 0/0
  Mix 14: 5/5
  Mix 16: 10/10
  Mix 22: 0/0
  Mix 26: 10/10
  Mix 27: 0/0
Accuracy: 100.0%

Day 2: 
  Mix 1: 0/0
  Mix 2: 0/0
  Mix 3: 12/12
  Mix 4: 4/4
  Mix 9: 0/0
  Mix 10: 6/6
  Mix 13: 0/0
  Mix 14: 5/5
  Mix 16: 10/10
  Mix 22: 0/0
  Mix 26: 10/10
  Mix 27: 10/10
Accuracy: 100.0%

Day 3: 
  Mix 1: 0/0
  Mix 2: 0/0
  Mix 3: 10/12
  Mix 4: 0/0
  Mix 9: 3/5
  Mix 10: 4/6
  Mix 13: 0/0
  Mix 14: 4/5
  Mix 16: 9/10
  Mix 22: 0/0
  Mix 26: 9/10
  Mix 27: 9/10
Accuracy: 82.75862068965517%

Day 4: 
  Mix 1: 0/0
  Mix 2: 0/0
  Mix 3: 11/12
  Mix 4: 0/0
  Mix 9: 4/5
  Mix 10: 5/6
  Mix 13: 0/0
  Mix 14: 5/5
  Mix 16: 10/10
  Mix 22: 0/0
  Mix 26: 10/10
  Mix 27: 0/0
Accuracy: 93.75%

Day 5: 
  Mix 1: 0/0
  Mix 2: 0/0
  Mix 3: 11/12
  Mix 4: 0/0
  Mix 9: 0/0
  Mix 10: 5/6
  Mix 13: 0/0
  Mix 14: 4/5
  Mix 16: 9/10
  Mix 22: 4/5
  Mix 26: 10/10
  Mix 27: 0/0
Accuracy: 89.58333333333333%

D

In [38]:
# Output current schedule
print("Output:\n[")
for i in range(len(schedule_copy)-1):
    print(end="[")
    for j in range(len(schedule_copy[i])-1):
        print(spices_indexes[schedule_copy[i][j]], end=", ")
    if i!=len(schedule_copy)-2:
        print(spices_indexes[schedule_copy[i][len(schedule_copy[i])-1]], end="],\n")
    else:
        print(spices_indexes[schedule_copy[i][len(schedule_copy[i])-1]], end="]\n")
print("]")

Output:
[
[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 2, 2, 2, 2, 2, 2, 2, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14],
[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 10, 10, 10, 10, 10, 10, 4, 4, 4, 4],
[27, 27, 27, 27, 27, 27, 27, 27, 27, 9, 9, 9, 10, 10, 10, 10, 16, 16, 16, 16, 16, 16, 16, 16, 16, 3, 3, 3, 3, 3, 3, 26, 26, 26, 26, 26, 26, 26, 26, 26, 14, 14, 14, 14],
[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 14, 14, 14, 14, 14, 9, 9, 9, 9, 10, 10, 10, 10, 10],
[16, 16, 16, 16, 16, 16, 16, 16, 16, 14, 14, 14, 14, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 22, 22, 22, 22, 10, 10, 10, 10, 10],
[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 26, 26, 26, 26, 26, 16,